In [ ]:
!pip install lightfm

In [8]:
import numpy as np
from lightfm.datasets import fetch_movielens
data = fetch_movielens()

ModuleNotFoundError: No module named 'lightfm'

In [9]:
for key, value in data.items():
    print(key, value.shape)

NameError: name 'data' is not defined

In [10]:
train = data['train']
test = data['test']

NameError: name 'data' is not defined

In [ ]:
from lightfm import LightFM
model = LightFM(learning_rate=0.05, loss='bpr')
model.fit(train, epochs=10)

In [1]:
from lightfm.evaluation import precision_at_k, auc_score
train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()
train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test, train_interactions=train).mean()
print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

ModuleNotFoundError: No module named 'lightfm'

In [ ]:
model = LightFM(learning_rate=0.05, loss='warp')
model.fit_partial(train, epochs=10)

In [2]:
train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()
train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test, train_interactions=train).mean()
print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

NameError: name 'precision_at_k' is not defined